[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mario-Carmona/ProyectoFinal-VC/blob/main/ProyectoFinal.ipynb)

# **Proyecto Final**